# 📊 구조화된 출력(Structured Output) 생성

> ⚠️ **알림**  
> **현재 올바로 동작하지 않는 이슈가 있습니다. 추후 업데이트 예정입니다**

## 🎯 개요

Azure OpenAI Chat Completion 서비스를 기반으로 구축된 에이전트를 사용하여 **구조화된 출력**을 생성하는 방법을 알아봅니다.

---

### 💡 중요 사항

#### 지원 에이전트 타입

모든 [에이전트 유형](SupportedAgentType.ipynb)이 구조화된 출력을 지원하는 것은 **아닙니다**.  
다음의 `ChatAgent` 유형은 다음의 호환 가능한 채팅 클라이언트와 함께 사용할 경우 구조화된 출력을 지원합니다:

✅ **AzureOpenAIChatClient** (Azure OpenAI Chat Completions)  
✅ **OpenAIResponsesClient** (OpenAI-compatible Responses API)  
✅ **GitHubOpenAIChatClient** (GitHub Models chat endpoint)

## 📝 Step 1: Pydantic 모델 정의

에이전트로부터 얻고자 하는 **출력의 구조**를 나타내는 Pydantic 모델을 정의합니다.

In [5]:
from pydantic import BaseModel

class PersonInfo(BaseModel):
    """사람에 대한 정보입니다. 👤ℹ️"""
    name: str | None = None
    age: int | None = None
    occupation: str | None = None

## 🤖 Step 2: 에이전트 생성

Azure OpenAI Chat 클라이언트를 사용하여 에이전트를 생성합니다.

In [6]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# Create the agent using Azure OpenAI Chat Client
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
   name="HelpfulAssistant",
   instructions="당신은 텍스트에서 사람 정보를 추출하는 도움이 되는 어시스턴트입니다. 📝✨"#
)


---

## ⚠️ Agent Framework의 Structured Outputs 버그

> 🛠️ **이하 부분은 추후 수정될 예정입니다. 랩에서는 수행하지 않습니다.**

### 🔍 문제점
Agent Framework의 `agent.run(response_format=PersonInfo)`가 `response.value`에 structured output을 반환하지 않습니다.

---

## 🎁 Step 3: 구조화된 데이터 접근

Response의 **`value`** 속성을 통해 PersonInfo 모델 인스턴스에 직접 액세스할 수 있습니다.

In [7]:
response = await agent.run(
    "Please provide information about John Smith, who is a 35-year-old software engineer.",
    response_format=PersonInfo
)
print("Response : " + str(response))

if response.value:
    person_info = response.value
    print(f"Name: {person_info.name}, Age: {person_info.age}, Occupation: {person_info.occupation}")
else:
    print("No structured data found in response")

Response : Name: John Smith  
Age: 35  
Profession: Software Engineer
No structured data found in response


## 🌊 Step 4: 스트리밍에서 구조화된 출력 처리

스트리밍 시 에이전트 응답은 **일련의 업데이트**로 스트리밍됩니다.  
구조화된 출력을 얻으려면 **모든 업데이트를 수집**한 다음 최종 응답 값에 액세스해야 합니다.

In [8]:
from agent_framework import AgentResponse

query = "Tell me about Jane Doe, a 28-year-old graphic designer."

# Get structured response from streaming agent using AgentResponse.from_agent_response_generator
# This method collects all streaming updates and combines them into a single AgentResponse
final_response = await AgentResponse.from_agent_response_generator(
    agent.run_stream(query, response_format=PersonInfo),
    output_format_type=PersonInfo,
)

if final_response.value:
    person_info = final_response.value
    print(f"Name: {person_info.name}, Age: {person_info.age}, Occupation: {person_info.occupation}")

Failed to parse value from agent run response text: 1 validation error for PersonInfo
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='**Name:** Jane Doe  \n**...ion:** Graphic Designer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid


ValidationError: 1 validation error for PersonInfo
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='**Name:** Jane Doe  \n**...ion:** Graphic Designer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid